In [1]:
import cv2 as cv
import tensorflow as tf
from IPython.display import clear_output
import matplotlib.pyplot as plt
import os
from google.colab.patches import cv2_imshow
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

In [2]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(3, 1, padding="same", activation="sigmoid")(d4)

    M = Model(inputs, outputs, name="U-Net")
    return M
  

In [3]:
input_shape = (128, 128, 3)
model = build_unet(input_shape)
model.summary()


Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [6]:
# Define training set and test set
c1= 0
c2= 0
img= []
result= []
filename= '/content/drive/MyDrive/UGV_AI/CVPPP2015_LCC_training_data/A1_img' + '/'
for i in os.listdir(filename):
    path = os.path.join(filename, i)
    i= Image.open(path)
    i= np.array(i)
    i= cv.cvtColor(i, cv.COLOR_BGR2RGB)
    #cv2_imshow(i)
    #print(np.shape(i))
    i= np.resize(i, (128, 384))
    #print(np.shape(i))
    i = np.reshape(i,(128,128,3)) 
    i = i/256
    c1= c1+1
    img.append(i)
    

filename= '/content/drive/MyDrive/UGV_AI/CVPPP2015_LCC_training_data/A1_result' + '/'
for j in os.listdir(filename):
    path = os.path.join(filename, j)
    j= Image.open(path)
    j= j.resize((128, 384))
    j= np.reshape(j,(128,128,3))
    j= j/256
    c2= c2+1
    result.append(j)

print(c1, c2)



128 128


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
X= np.array(img, np.float32)
y= np.array(result, np.float32)

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.nn.sigmoid_cross_entropy_with_logits, metrics=['accuracy'])
results = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_valid, y_valid))
model.summary(line_length=None, positions=None, print_fn=None, expand_nested=False)

Epoch 1/20
3/3 [==============================] - 171s 55s/step - loss: 0.9678 - accuracy: 0.3539 - val_loss: 1.1321 - val_accuracy: 0.2610
Epoch 2/20
3/3 [==============================] - 166s 55s/step - loss: 0.8843 - accuracy: 0.2682 - val_loss: 1.2654 - val_accuracy: 0.9476
Epoch 3/20
3/3 [==============================] - 165s 55s/step - loss: 0.8483 - accuracy: 0.3512 - val_loss: 1.2223 - val_accuracy: 0.9581
Epoch 4/20
3/3 [==============================] - 167s 56s/step - loss: 0.8314 - accuracy: 0.2964 - val_loss: 1.2067 - val_accuracy: 0.9665
Epoch 5/20
3/3 [==============================] - 166s 55s/step - loss: 0.8161 - accuracy: 0.2561 - val_loss: 1.1632 - val_accuracy: 0.9175
Epoch 6/20
3/3 [==============================] - 166s 55s/step - loss: 0.8059 - accuracy: 0.3022 - val_loss: 1.1556 - val_accuracy: 0.9187
Epoch 7/20
3/3 [==============================] - 166s 55s/step - loss: 0.7973 - accuracy: 0.4477 - val_loss: 1.1578 - val_accuracy: 0.9473
Epoch 8/20
3/3 [====